## Import data 

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import shapiro 

from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import mean_absolute_error

import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.graphics.tsaplots import plot_predict

import seaborn as sns

import colorama
from colorama import Fore, Style

! pip install pmdarima
from pmdarima import auto_arima

In [ ]:
df = pd.read_csv('Natural Gas Consumption at disaggregate level.csv',delimiter=';')

df.head()

## Data Preprocessing and Visualization 

In [ ]:
#reorganize data columns
df1 = pd.melt(df, id_vars=["Country"], var_name="Year", value_name="Value")

df1.shape

In [ ]:
df1.head()

In [ ]:
#Eliminate the countries which are not in European Union
values = ['Iceland','Norway','Switzerland','Turkey','Ukraine', 'United Kingdom', 'Other Europe']
df1 = df1[df1.Country.isin(values) == False]
df1

In [ ]:
#convert datatype from string to float
df1["Value"] =[float(str(i).replace(',','.')) for i in df1["Value"]] 

In [ ]:
df1.dtypes #check the data types

In [ ]:
#See whether the dataset has any null value
df1.isnull().sum()

In [ ]:
#pivot feature to see the visualization
df1 = df1.pivot_table('Value', ['Year'], 'Country').reset_index()

In [ ]:
#Covert Year to date column
df1['Year'] = pd.to_datetime(df1['Year'])

In [ ]:
#create a country list
country_name=np.array(df1.columns.unique())
country_name = np.delete(country_name, 0)
print(country_name)

In [ ]:
#the info of dataset
df1.info()

In [ ]:
# To compelte the data, as naive method, we will use ffill
f, ax = plt.subplots(nrows=len(country_name), ncols=1,  figsize=(12,len(country_name)*3.5))
for i, column in enumerate(df1.drop('Year', axis=1).columns):
    sns.lineplot(x=df1['Year'], y=df1[column].fillna(method='ffill'), ax=ax[i], color='dodgerblue')
    ax[i].set_title('{}'.format(column), fontsize=14)
    ax[i].set_ylabel(ylabel='Natural Gas consumption at disaggregate level by Year', fontsize=8)
    ax[i].set_xlabel('Year', size=10)
    ax[i].xaxis.set_tick_params(labelsize=9)
    ax[i].yaxis.set_tick_params(labelsize=9)
plt.tight_layout() 

## Handle Missings and Descriptive statistics

In [ ]:
df1 = pd.melt(df1, id_vars=["Year"], var_name="Country", value_name="Value")

df1.shape

In [ ]:
df1["Value"] =[float(str(i).replace(',','.')) for i in df1["Value"]] 

In [ ]:
# fill null data with the mean of each group

df1['Value']= df1.groupby('Country')['Value'].apply(lambda x: x.fillna(x.mean()))

In [ ]:
#round up after comma
df1['Value']=round(df1['Value'],2) 
df1.head()

In [ ]:
df1 = df1.pivot_table('Value', ['Year'], 'Country').reset_index()

In [ ]:
# The descriptive statistic information for each country
df1_describe=df1.describe()
df1_describe.loc['var'] = df1.var().tolist() #variance
df1_describe.loc['skew'] = df1.skew().tolist() #skewness
df1_describe.loc['kurt'] = df1.kurtosis().tolist() #kurtosis

In [ ]:
my_natural_gas_data_description=df1_describe.T
my_natural_gas_data_description.to_csv("my_natural_gas_data_description.csv") #save as csv file

# Stationarity

## Augmented Dickey-Fuller (ADF)

In [ ]:
#ADF test to check the stationarity of each country's data
for name in country_name:
    def adf_test(value):
        result=adfuller(value)
        labels = ['Test parameters', 'p-value','#Lags Used','Dataset observations']
        for value,label in zip(result,labels):
            print(label+' : '+str(value) )
        if result[1] <= 0.05:
            print("Dataset is stationary")
            print(name)
            print("\n")
        else:
            print("Dataset is non-stationary ")
            print(name)
            print("\n")
    adf_test(df1[name])

## Normality test:

In [ ]:
df1 = pd.melt(df1, id_vars=["Year"], 
                  var_name="Country", value_name="Value")
df1.head()

In [ ]:
my_normality_test=(df1.groupby('Country')[['Value']].apply(lambda x: pd.Series(shapiro(x), index=['Shapiro-Wilk','P'])).reset_index())
my_normality_test.to_csv("my_normality_test_natural_gas_consumption.csv") #save as csv file

## Plotting Autocorrelation Function (ACF) for seasonality inspection

In [ ]:
#Autocorrelation visualization to check whether the data of each country is stationary
fig, ax = plt.subplots(nrows=len(country_name), figsize=(12,len(country_name)*3.5))
for i in country_name:
    #print(i)
    ind =list(country_name).index(i)
    #print(ind)
    filt = df1['Country']== i 
    #print(filt)
    sm.graphics.tsa.plot_acf(df1.loc[filt]['Value'],ax=ax[ind])
    ax[ind].set_title(i, size=10)
    ax[ind].set_ylabel('Natural Gas Consumption by Year', size=9)
    ax[ind].set_xlabel('Year', size=9)
    ax[ind].xaxis.set_tick_params(labelsize=9)
    ax[ind].yaxis.set_tick_params(labelsize=9)

plt.tight_layout() 

## Implementing Differencing

In [ ]:
#1st order differencing segment of ARIMA makes the data stationary.

df1['Value_diff'] = df1.groupby(['Country'])['Value'].diff().fillna(0)

In [ ]:
#ADF test for checking the stationarity of data
for name in country_name:
    X = df1[df1['Country'] == name]['Value_diff'].values
    def adf_test(value):
        result=adfuller(value)
        labels = ['Test parameters', 'p-value','#Lags Used','Dataset observations']
        for value,label in zip(result,labels):
            print(label+' : '+str(value) )
        if result[1] <= 0.05:
            print("Dataset is stationary")
            print(name)
            print("\n")
        else:
            print("Dataset is non-stationary ")
            print(name)
            print("\n")
    adf_test(X)

In [ ]:
#2nd order differencing segment of ARIMA makes the data stationary.
df1['Value_diff_1'] = df1.groupby(['Country'])['Value_diff'].diff().fillna(0)

In [ ]:
#ADF test for checking the stationarity of data
for name in country_name:
    X = df1[df1['Country'] == name]['Value_diff_1'].values
    def adf_test(value):
        result=adfuller(value)
        labels = ['Test parameters', 'p-value','#Lags Used','Dataset observations']
        for value,label in zip(result,labels):
            print(label+' : '+str(value) )
        if result[1] <= 0.05:
            print("Dataset is stationary")
            print(name)
            print("\n")
        else:
            print("Dataset is non-stationary ")
            print(name)
            print("\n")
    adf_test(X)

In [ ]:
#2nd order differencing segment of ARIMA makes the data stationary.
df1['Value_diff_2'] = df1.groupby(['Country'])['Value_diff_1'].diff().fillna(0)

In [ ]:
#ADF test for checking the stationarity of data
for name in country_name:
    X = df1[df1['Country'] == name]['Value_diff_2'].values
    def adf_test(value):
        result=adfuller(value)
        labels = ['Test parameters', 'p-value','#Lags Used','Dataset observations']
        for value,label in zip(result,labels):
            print(label+' : '+str(value) )
        if result[1] <= 0.05:
            print("Dataset is stationary")
            print(name)
            print("\n")
        else:
            print("Dataset is non-stationary ")
            print(name)
            print("\n")
    adf_test(X)

## Plotting ACF

In [ ]:
# Plotting ACF to get the best value of q.
fig, ax = plt.subplots(nrows=len(country_name), figsize=(12,len(country_name)*3.5))
for i in country_name:
    #print(i)
    ind =list(country_name).index(i)
    #print(ind)
    filt = df1['Country']== i 
    #print(filt)
    sm.graphics.tsa.plot_acf(df1.loc[filt]['Value_diff_2'],ax=ax[ind], lags=45)
    ax[ind].set_title(i, size=10)
    ax[ind].set_ylabel('Natural Gas Consumption by Year', size=9)
    ax[ind].set_xlabel('Year', size=9)
    ax[ind].xaxis.set_tick_params(labelsize=9)
    ax[ind].yaxis.set_tick_params(labelsize=9)
plt.tight_layout() 

## Plotting the new dataset

In [ ]:
#Plotting the new dataset to the stationarity
fig, ax = plt.subplots(nrows=len(country_name), figsize=(12,len(country_name)*3.5))
for i in country_name:
    #print(i)
    ind =list(country_name).index(i)
    #print(ind)
    filt = df1['Country']== i 
    #print(filt)
    ax[ind].plot(df1.loc[filt]['Year'],df1.loc[filt]['Value_diff_2'])
    ax[ind].set_title(i, size=10)
    ax[ind].set_ylabel('Natural Gas Consumption by Year', size=9)
    ax[ind].set_xlabel('Year', size=9)
    ax[ind].xaxis.set_tick_params(labelsize=9)
    ax[ind].yaxis.set_tick_params(labelsize=9)
plt.tight_layout() 

## Plotting PACF

In [ ]:
#Plotting PACF to get the best value of p
fig, ax = plt.subplots(nrows=len(country_name), figsize=(12,len(country_name)*3.5))
for i in country_name:
    #print(i)
    ind =list(country_name).index(i)
    #print(ind)
    filt = df1['Country']== i 
    #print(filt)
    sm.graphics.tsa.plot_pacf(df1.loc[filt]['Value_diff_2'],ax=ax[ind],lags=22)
    ax[ind].set_title(i, size=10)
    ax[ind].set_ylabel('Natural Gas Consumption by Year', size=9)
    ax[ind].set_xlabel('Year', size=9)
    ax[ind].xaxis.set_tick_params(labelsize=9)
    ax[ind].yaxis.set_tick_params(labelsize=9)
plt.tight_layout() 

## Training and Testing the ARIMA model - Accuracy Metrics for Time Series Forecast

In [ ]:
warnings.filterwarnings('ignore')
# Fitting an ARIMA model and Interpret the result of an ARIMA model
warnings.filterwarnings('ignore')
dict_org = {}
dict_pred = {}
country_accuracy = {}
for name in range(len(country_name)):
    X = df1[df1['Country'] == country_name[name]]['Value'].values
    size = int(len(X) * 0.70)
    train, test = X[0:size], X[size:len(X)]
    history =[x for x in train]
    #print(train.shape,test.shape)
    predictions = list()
    for t in range(len(test)):
        model = sm.tsa.arima.ARIMA(history, order = (2,3,1))
        model_fit = model.fit()
        output = model_fit.forecast()
        yhat = output[0]
        predictions.append(yhat)
        obs = test[t]
        history.append(obs)
        print(Fore.BLUE +"Predicted:%f, expected:%f" %(yhat, obs))
        
        #the residuals to ensure there are no patterns
        residuals = pd.DataFrame(model_fit.resid)
    #Accuracy Metrics for Time Series Forecast 
    mse= mean_squared_error(test, predictions) #Mean Squared Error
    mae=mean_absolute_error(test, predictions) #Mean Absolute Error
    rmse=sqrt(mse) #RMSE
    mape=np.mean(np.abs(predictions-test)/np.abs(test)) #Mean absolute percentage error
    corr = np.corrcoef(predictions,test)[0,1] #Correlation between the Actual and the Forecast (corr)
  
    dict_org.update({country_name[name]: test})
    dict_pred.update({country_name[name]: predictions})
    print('Accuracy Metrics:')
    print("Group: ", country_name[name], "Test MSE:%f"% mse)
    print("Group: ", country_name[name], "Test MAE:%f"% mae)
    print("Group: ", country_name[name], "Test RMSE:%f"% rmse)
    print("Group: ", country_name[name], "Test MAPE:%f"% mape)
    print("Group: ", country_name[name], "Test CORR:%f"% corr)
    country_accuracy.update({country_name[name]: mse,country_name[name]: rmse,country_name[name]: mape,country_name[name]: mae,country_name[name]: corr})
    #Plot test and predictions data
    plt.plot(test)
    plt.plot(predictions, color = 'orange')
    plt.legend(['Actual','Predicted'])
    plt.xlabel(country_name[name])
    plt.show()
    # Plot residual errors
    residuals.plot(title='Residuals for the data of '+country_name[name])
    plt.show()
    residuals.plot(kind='kde', title='Density for the data of '+country_name[name])
    plt.show()

## Auto ARIMA model- Model Checking

In [ ]:
dict_original = {}
dict_prediction = {}
country_accuracy = {}
for name in range(len(country_name)):
    X = df1[df1['Country'] == country_name[name]]['Value'].values
    size = int(len(X) * 0.70)
    train, test = X[0:size], X[size:len(X)]
    history =[x for x in train]
    #print(train.shape,test.shape)
    predictions = list()
    for t in range(len(test)):
        model =auto_arima(history, start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True) # this works 
        output = model.predict()
        yhat = output[0]
        predictions.append(yhat)
        obs = test[t]
        history.append(obs)
        print(Fore.BLUE +"Predicted:%f, expected:%f" %(yhat, obs))
        
        
    mse= mean_squared_error(test, predictions) #Mean Squared Error
    mae=mean_absolute_error(test, predictions) #Mean Absolute Error
    rmse=sqrt(error) #RMSE
    mape=np.mean(np.abs(predictions-test)/np.abs(test)) #Mean absolute percentage error
    print('Test MSE: %.3f' % error)
    dict_org.update({country_name[name]: test})
    dict_pred.update({country_name[name]: predictions})
    print("Group: ", country_name[name], "Test MSE:%f"% mse)
    print("Group: ", country_name[name], "Test MAE:%f"% mae)
    print("Group: ", country_name[name], "Test RMSE:%f"% rmse)
    print("Group: ", country_name[name], "Test MAPE:%f"% mape)
    country_accuracy.update({country_name[name]: mse,country_name[name]: rmse,country_name[name]: mape,country_name[name]: mae})
    
    #Plotting
    plt.plot(test)
    plt.plot(predictions, color = 'orange')
    plt.legend(['Actual','Predicted'])
    plt.xlabel(country_name[name])
    plt.show()